In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_10.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_11.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_12.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_13.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_14.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_15.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_16.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_17.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_18.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_19.png  
  inflating: Dataset/Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_20.png  

####**Importing Dependencies**

In [ ]:
import tensorflow 
from tensorflow import keras
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Conv2D,Flatten, MaxPooling2D,BatchNormalization,Dropout

####**Model Building**

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='valid', activation='relu',input_shape=(224,224,3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='valid', activation='relu'))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='valid', activation='relu'))
model.add(MaxPooling2D(2,2))

#model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(500, activation= 'relu'))

model.add(Dense(300, activation= 'relu'))
model.add(Dropout(0.1))

model.add(Dense(1, activation= 'sigmoid'))

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_33 (Conv2D)          (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 111, 111, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 54, 54, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_35 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_35 (MaxPoolin  (None, 26, 26, 128)    

In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy", metrics=["accuracy"])

####**Building Train and Test Image generator**

In [ ]:
def traingenerator(path):
    image_data= ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255, horizontal_flip=True,vertical_flip= True)

    image= image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,shuffle= True,class_mode= 'binary')

    return image

In [ ]:
train_image= traingenerator('/content/Dataset/Train')

Found 416 images belonging to 2 classes.


In [ ]:
def testgenerator(path):
    image_data= ImageDataGenerator(rescale=1/255)

    image= image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,shuffle= True,class_mode= 'binary')

    return image

In [ ]:
test_image= testgenerator('/content/Dataset/Test')

Found 134 images belonging to 2 classes.


####**Training The Model**

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

es= EarlyStopping(monitor='accuracy',min_delta=0.01,patience=5,verbose=1,mode='auto')

mc= ModelCheckpoint(monitor='accuracy',save_best_only= True,filepath= '/content/bestmodel.h5',verbose=1,mode='auto')

cb= [es,mc]

In [ ]:
model.fit(train_image,steps_per_epoch= len(train_image),epochs= 50,callbacks=cb)

Epoch 1/50
13/13 [==============================] - ETA: 0s - loss: 0.1770 - accuracy: 0.9519
Epoch 1: accuracy improved from -inf to 0.95192, saving model to /content/bestmodel.h5
13/13 [==============================] - 8s 578ms/step - loss: 0.1770 - accuracy: 0.9519
Epoch 2/50
13/13 [==============================] - ETA: 0s - loss: 0.1262 - accuracy: 0.9567
Epoch 2: accuracy improved from 0.95192 to 0.95673, saving model to /content/bestmodel.h5
13/13 [==============================] - 7s 516ms/step - loss: 0.1262 - accuracy: 0.9567
Epoch 3/50
13/13 [==============================] - ETA: 0s - loss: 0.1660 - accuracy: 0.9712
Epoch 3: accuracy improved from 0.95673 to 0.97115, saving model to /content/bestmodel.h5
13/13 [==============================] - 7s 515ms/step - loss: 0.1660 - accuracy: 0.9712
Epoch 4/50
13/13 [==============================] - ETA: 0s - loss: 0.2010 - accuracy: 0.9327
Epoch 4: accuracy did not improve from 0.97115
13/13 [==============================] - 5s

In [ ]:
from keras.models import load_model

model = load_model('/content/bestmodel.h5')

In [ ]:
model.evaluate(test_image)

5/5 [==============================] - 0s 50ms/step - loss: 0.1308 - accuracy: 0.9478


[0.13080060482025146, 0.9477611780166626]